In [3]:
import json
import sys
import os
from urllib.parse import quote, unquote
import re
sys.path.append("/Users/v685573/Documents/Development/vzrisk/flow/")  # where rdfaf is
import rdfaf

In [4]:
NAMESPACE = "https://vz-risk.github.io/flow/attack-flow#"
CASEFILES = ['/Users/v685573/Documents/mitre/attack flow/attack-flow/corpus/tesla.json',
             '/Users/v685573/Documents/mitre/attack flow/attack-flow/corpus/cobalt-kitty-attack-flow.json',
             '/Users/v685573/Documents/mitre/attack flow/attack-flow/corpus/dfir_report_zero_to_domain_admin.json'
            ]
CASEFILE = CASEFILES[2]
AF_SCHEMA_JSONSCHEMA = "/Users/v685573/Documents/mitre/attack flow/attack-flow/schema/attack-flow-2022-01-05-draft.json"
AF_SCHEMA_JSONLD = "/Users/v685573/Documents/Development/vzrisk/flow/attack_flow.jsonld"
OUTDIR = "/Users/v685573/Documents/Development/vzrisk/flow/corpus/"

In [5]:
### We have to do this to produce a consistent namespace for the flow itself.
### The flow should be in an absolute namespace associated with the victim.

# Create a unique namespace
incident_namespace = "urn:absolute:{0}#".format(os.path.splitext(os.path.split(CASEFILE)[-1])[0])
incident_namespace = re.sub("[^0-9a-zA-Z_.\-~]+", "_", incident_namespace)
incident_namespace = quote(incident_namespace)

with open(CASEFILE, 'r') as filehandle:
    cfj = json.load(filehandle)
cfj['flow']['id'] = NAMESPACE + cfj['flow']['id'].split("//")[-1].split("#")[-1]
# The CK URIs are a bit messy so moving into the same namespace
for i in range(len(cfj['actions'])):
    action = cfj['actions'][i]
    action['id'] = NAMESPACE + action['id'].split("//")[-1].split("#")[-1]
    cfj['actions'][i] = action
for i in range(len(cfj['assets'])):
    asset = cfj['assets'][i]
    asset['id'] = NAMESPACE + asset['id'].split("//")[-1].split("#")[-1]
    cfj['assets'][i] = asset
for i in range(len(cfj['relationships'])):
    rel = cfj['relationships'][i]
    rel['source'] = NAMESPACE + rel['source'].split("//")[-1].split("#")[-1]
    rel['type'] = NAMESPACE + rel['type'].split("//")[-1].split("#")[-1]
    rel['target'] = NAMESPACE + rel['target'].split("//")[-1].split("#")[-1]
    cfj['relationships'][i] = rel
for i in range(len(cfj['object_properties'])):
    prop = cfj['object_properties'][i]
    prop['source'] = NAMESPACE + prop['source'].split("//")[-1].split("#")[-1]
    prop['type'] = NAMESPACE + prop['type'].split("//")[-1].split("#")[-1]
    prop['target'] = NAMESPACE + prop['target'].split("//")[-1].split("#")[-1]
    cfj['object_properties'][i] = prop
for i in range(len(cfj['data_properties'])):
    prop = cfj['data_properties'][i]
    prop['source'] = NAMESPACE + prop['source'].split("//")[-1].split("#")[-1]
    prop['type'] = NAMESPACE + prop['type'].split("//")[-1].split("#")[-1]
    prop['target'] = NAMESPACE + prop['target'].split("//")[-1].split("#")[-1]
    cfj['data_properties'][i] = prop

In [6]:
###

# create the flow object
case = rdfaf_v2_6_1.flow()
# load the namespace
case.load_namespace(NAMESPACE)

with open(AF_SCHEMA_JSONSCHEMA, 'r') as filehandle:
    case.load_schema(json.load(filehandle))

rules = rdfaf_v2_6_1.Graph()
rules.parse(AF_SCHEMA_JSONLD, format="json-ld")
case.load_rules(rules)
# load the flow
case.load_flow(cfj)

In [7]:
filename = os.path.splitext(os.path.split(CASEFILE)[-1])[0]
fileExt = ".jsonld"
with open(OUTDIR + filename + fileExt, 'w') as filehandle:
    filehandle.write(case.get_flow_graph(rules=False).serialize(format="json-ld"))